<a href="https://colab.research.google.com/github/MikeHu13/CV_V5/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow import keras
from tensorflow.keras import Model
# setting up google drive
import platform
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
# set the correct root dir (precreate on google drive)
root_dir = 'gdrive/My Drive/set_small/'



# Import the chosen network
conv_base = keras.applications.InceptionResNetV2( # Xception ...
is actual top one
weights='imagenet', # Load weights pre-trained on ImageNet.
input_shape=(150, 150, 3),
include_top=False) # Do not include the ImageNet classifier at ...
the top.
# Print strucutre in console
conv_base.summary()


In [ ]:
# second part
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()



# Output to console
print('This is the number of trainable weights '
'before freezing the conv base:',)
model.summary()


# Freeze the conv base: set switch to freeze
conv_base.trainable = False
# Output to compare
print('This is the number of trainable weights '
'after freezing the conv base:')
model.summary()


In [ ]:
# load modules needed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# set directories for training, validation and test set, here ...
google drive
base_dir = 'gdrive/My Drive/set_small/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
# Set up constructor for data augmentation
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')
# Test set only normalized, not augmented
test_datagen = ImageDataGenerator(rescale=1./255)
# Collect the images and do the augmentation, labels are set to binary
# augment training data
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
# augment validation data
validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')


In [ ]:
# Elseif cases with fall back
if TRANSFER:
print('Using Transfer learning...')
# parameters for model training
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(learning_rate=2e-5),
metrics=['acc'])
# do the training
history = model.fit(
train_generator,
steps_per_epoch=100, # should be num_train/batch_size
epochs=30,
validation_data=validation_generator,
validation_steps=50) # should be num_val/batch_size
elif FINETUNE:
print('Finetuning the model...')
# set the parameters, here only freeze blocks 1 to 4
# The loop freezes all layers UP TO a specific one
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
# Replace according to your architecture...
#if layer.name == 'block5_conv1':
if layer.name == 'conv5_block3_3_conv':
set_trainable = True
if set_trainable:
layer.trainable = True
else:
layer.trainable = False
# compile the model with given parameters
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])
# Fine tune the model and start training
history = model.fit(
train_generator,
steps_per_epoch=100,
epochs=100,
validation_data=validation_generator,
validation_steps=50)
# Fallback if something wents wrong
else:
print('No freeze model, no other model')
# save the trained model
save_dir = os.path.join(root_dir, 'cats_dogs_net.h5')
model.save(save_dir)


In [ ]:
# get the recorded data for training and validation looking at ...
accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
# plot the curves
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()



# test the accuraccy
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
# evaluate the test images with the model
test_loss, test_acc = model.evaluate(test_generator, steps=50)
# Output to console
print('test acc:', test_acc)
